In [1]:
import numpy as np
import util
from ANN import *
from tensorboardX import SummaryWriter
import time
from ANN2 import *

ModuleNotFoundError: No module named 'matplotlib'

## 1. Data Loading 

In [ ]:
classification_train_data = util.load_data("./data/classification_train_data.pkl")
classification_train_label = util.load_data("./data/classification_train_label.pkl")
classification_test_data = util.load_data("./data/classification_test_data.pkl")

## 2. Data Exploration

In [ ]:
print("Classification Train Data Shape:", classification_train_data.shape)
print("Classification Train Label Shape:", classification_train_label.shape)
print("Classification Test Data Shape:", classification_test_data.shape)

Classification Train Data Shape: (49976, 257)
Classification Train Label Shape: (49976, 2)
Classification Test Data Shape: (9997, 257)


## 3. Data Preprocessing

In [ ]:
# remove index column
train_data_index = classification_train_data[:, 0]
train_label_index = classification_train_label[:, 0]
test_data_index = classification_test_data[:, 0]
classification_train_data = classification_train_data[:, 1:]
classification_train_label = classification_train_label[:, 1:].reshape(-1)
classification_test_data = classification_test_data[:, 1:]

In [ ]:
classification_train_data.shape, classification_train_label.shape, classification_test_data.shape

((49976, 256), (49976,), (9997, 256))

In [ ]:
train_data_index.shape, train_label_index.shape, test_data_index.shape

((49976,), (49976,), (9997,))

In [ ]:
# normalization

# calculate the mean and standard deviation of each column
mean = np.mean(classification_train_data, axis=0)
std_dev = np.std(classification_train_data, axis=0)

# Z-Score normalizes each column
classification_train_data = (classification_train_data - mean) / std_dev
classification_test_data = (classification_test_data - mean) / std_dev

In [ ]:
# label one-hot encoding
num_classes =  10 
classification_train_label = np.eye(num_classes)[classification_train_label]
print("train label shape:", classification_train_label.shape)

train label shape: (49976, 10)


## 4. Dataset Splitting

In [ ]:
# divide the data set into training set and validation set
train_ratio = 0.8
seed = 123
(train_data, train_labels), (validation_data, validation_labels) = util.split_train_validation(
    classification_train_data, classification_train_label,
    train_ratio=train_ratio, random_seed=seed
    )

In [ ]:

train_set = (train_data, train_labels)
val_set = (validation_data, validation_labels)
train_data.shape, train_labels.shape, validation_data.shape, validation_labels.shape


((39980, 256), (39980, 10), (9996, 256), (9996, 10))

# 5. Model

In [ ]:
ann = MyANN(n_inputs=256,n_hidden=16,n_outputs=10)
ann2 = NeuralNetwork(n_inputs=256,n_hidden=8,n_outputs=10,dropout_rate=0.2)


In [ ]:
train_wr = SummaryWriter(log_dir='logs/Train')
val_wr = SummaryWriter(log_dir='logs/Val')

## 6. Train 

In [ ]:
def random_shuffle(data,label):
    randnum = np.random.randint(0, 1234)
    np.random.seed(randnum)
    np.random.shuffle(data)
    np.random.seed(randnum)
    np.random.shuffle(label)
    return data,label

def train_network(model:NeuralNetwork, train_set:tuple, val_set:tuple, opt:My_Opt, n_epoch, save_path, loss_func):
    train_size=len(train_set[0])
    val_size=len(val_set[0])
    train_x, train_y = train_set
    val_x, val_y = val_set
    best_val = 0.0
    for epoch in range(n_epoch):
        train_pred=[]
        train_loss = 0
        learning_rate = opt.get_learning_rate()
        train_x, train_y = random_shuffle(train_x, train_y)
        val_x, val_y = random_shuffle(val_x, val_y)
        for index in tqdm(range(train_size)):
            row, expected = train_x[index], train_y[index]
            outputs = model.forward(row)
            train_pred.append(outputs)
            train_loss += loss_func(outputs, expected)
            model.backward(row, expected, learning_rate)
        opt.step()
        train_accuracy = accuracy_score(np.argmax(train_y, axis=1), np.argmax(train_pred, axis=1))
        train_wr.add_scalar('accuracy', train_accuracy, epoch)
        train_wr.add_scalar('loss', train_loss, epoch)


        val_pred=[]
        val_loss = 0
        for index in tqdm(range(val_size)):
            row, expected = val_x[index], val_y[index]
            outputs = model.predict(row)
            val_pred.append(outputs)
            val_loss += loss_func(outputs, expected)
        val_accuracy = accuracy_score(np.argmax(val_y, axis=1), np.argmax(val_pred, axis=1))
        val_wr.add_scalar('accuracy', val_accuracy, epoch)
        val_wr.add_scalar('loss', val_loss, epoch)
        if (val_accuracy > best_val):
            best_val = val_accuracy
            util.save_data(save_path, model)
            


        print('>epoch=%d, learning_rate=%.3f, train_acc=%.4f, val_acc=%.4f' % (epoch, learning_rate, train_accuracy,val_accuracy))

In [ ]:
loss_func=cross_entropy_loss
my_opt = My_Opt(alpha=0.5, beta1=0.9, beta2=0.999, epsilon=1e-8)

# train_network(model=ann,train_set=train_set, val_set=val_set, opt=my_opt,
#               n_epoch=30, save_path='./pths/model.pth', loss_func=cross_entropy_loss)


In [ ]:
train_network(model=ann2,train_set=train_set, val_set=val_set, opt=my_opt, n_epoch=30, save_path='./pths/model2.pth', loss_func=cross_entropy_loss)


 15%|█▍        | 5882/39980 [00:00<00:01, 29117.42it/s]

100%|██████████| 9996/9996 [00:00<00:00, 68902.14it/s]

Saved successfully


>epoch=0, learning_rate=0.158, train_acc=0.1348, val_acc=0.1612


100%|██████████| 9996/9996 [00:00<00:00, 71836.44it/s]

...

>epoch=27, learning_rate=0.088, train_acc=0.4626, val_acc=0.4431


100%|██████████| 9996/9996 [00:00<00:00, 68764.04it/s]

>epoch=28, learning_rate=0.089, train_acc=0.4619, val_acc=0.4436


100%|██████████| 9996/9996 [00:00<00:00, 70464.78it/s]

Saved successfully
>epoch=29, learning_rate=0.090, train_acc=0.4637, val_acc=0.4504


## 7. Predict

In [ ]:
test_label_predict = ann.predict(classification_test_data)

In [ ]:
# merge index and corresponding classification results 
submit_data = np.hstack((
    test_data_index.reshape(-1, 1),
    test_label_predict.reshape(-1, 1)
    ))

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
submit_data.shape

In [ ]:

util.save_data('./classification_results.pkl', submit_data)